In [1]:
import os
import sys
import tensorflow as tf
import numpy as np
import amlrealtimeai
from amlrealtimeai import resnet50

In [2]:
import glob
import imghdr
datadir = os.path.expanduser("/Users/atsaris/mygit_repos/ml_service/datasets")

cat_files = glob.glob(os.path.join(datadir, 'PetImages', 'Cat', '*.jpg'))
dog_files = glob.glob(os.path.join(datadir, 'PetImages', 'Dog', '*.jpg'))

# Limit the data set to make the notebook execute quickly.
cat_files = cat_files[:64]
dog_files = dog_files[:64]

# The data set has a few images that are not jpeg. Remove them.
cat_files = [f for f in cat_files if imghdr.what(f) == 'jpeg']
dog_files = [f for f in dog_files if imghdr.what(f) == 'jpeg']

In [3]:
# constructing a numpy array as labels
image_paths = cat_files + dog_files
total_files = len(cat_files) + len(dog_files)
labels = np.zeros(total_files)
labels[len(cat_files):] = 1

In [4]:
# Input images as a two-dimensional tensor containing an arbitrary number of images represented a strings
import amlrealtimeai.resnet50.utils
in_images = tf.placeholder(tf.string)
image_tensors = resnet50.utils.preprocess_array(in_images)
print(image_tensors.shape)

(?, 224, 224, 3)


In [5]:
subscription_id = "80defacd-509e-410c-9812-6e52ed6a0016"
resource_group = "CMS_FPGA_Resources"
model_management_account = "CMS_FPGA_1"

from amlrealtimeai.resnet50.model import RemoteQuantizedResNet50
model_path = os.path.expanduser('~/models')
featurizer = RemoteQuantizedResNet50(subscription_id, resource_group, model_management_account, model_path)
print(featurizer.version)

Using TensorFlow backend.


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code E3HT8PL27 to authenticate.
1.1.6-rc


In [6]:
featurizer.import_graph_def(include_top=False, input_tensor=image_tensors)
features = featurizer.featurizer_output

Registering model resnet50-1.1.6-rc-model
Successfully registered model resnet50-1.1.6-rc-model
Creating service featurizer-service-08cdb9
. . . . . . . 
Successfully created service featurizer-service-08cdb9


In [8]:
from tqdm import tqdm

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

def read_files(files):
    contents = []
    for path in files:
        with open(path, 'rb') as f:
            contents.append(f.read())
    return contents
        
feature_list = []
with tf.Session() as sess:
    for chunk in tqdm(chunks(image_paths, 5)):
        contents = read_files(chunk)
        result = sess.run([features], feed_dict={in_images: contents})
        feature_list.extend(result[0])

feature_results = np.array(feature_list)
print(feature_results.shape)

26it [00:13,  1.89it/s]

(128, 1, 1, 2048)


In [9]:
from keras.models import Sequential
from keras.layers import Dropout, Dense, Flatten
from keras import optimizers

FC_SIZE = 1024
NUM_CLASSES = 2

model = Sequential()
model.add(Dropout(0.2, input_shape=(1, 1, 2048,)))
model.add(Dense(FC_SIZE, activation='relu', input_dim=(1, 1, 2048,)))
model.add(Flatten())
model.add(Dense(NUM_CLASSES, activation='sigmoid', input_dim=FC_SIZE))

model.compile(optimizer=optimizers.SGD(lr=1e-4,momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
from sklearn.model_selection import train_test_split
onehot_labels = np.array([[0,1] if i else [1,0] for i in labels])
X_train, X_test, y_train, y_test = train_test_split(feature_results, onehot_labels, random_state=42, shuffle=True)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(96, 1, 1, 2048) (32, 1, 1, 2048) (96, 2) (32, 2)


In [11]:
model.fit(X_train, y_train, epochs=16, batch_size=32)

Epoch 1/16
96/96 [==============================] - 0s 2ms/step - loss: 0.7973 - acc: 0.5469
Epoch 2/16
96/96 [==============================] - 0s 348us/step - loss: 0.8021 - acc: 0.5260
Epoch 3/16
96/96 [==============================] - 0s 400us/step - loss: 0.7748 - acc: 0.5521
Epoch 4/16
96/96 [==============================] - 0s 372us/step - loss: 0.7173 - acc: 0.5625
Epoch 5/16
96/96 [==============================] - 0s 384us/step - loss: 0.7384 - acc: 0.5469
Epoch 6/16
96/96 [==============================] - 0s 372us/step - loss: 0.7059 - acc: 0.6354
Epoch 7/16
96/96 [==============================] - 0s 382us/step - loss: 0.6052 - acc: 0.6979
Epoch 8/16
96/96 [==============================] - 0s 355us/step - loss: 0.6177 - acc: 0.6771
Epoch 9/16
96/96 [==============================] - 0s 382us/step - loss: 0.5627 - acc: 0.7188
Epoch 10/16
96/96 [==============================] - 0s 341us/step - loss: 0.5484 - acc: 0.7656
Epoch 11/16
96/96 [==============================] 

In [12]:
from numpy import argmax

y_probs = model.predict(X_test)
y_prob_max = np.argmax(y_probs, 1)
y_test_max = np.argmax(y_test, 1)
print(y_prob_max)
print(y_test_max)

[0 0 0 0 1 0 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 0 0 0 0]
[0 0 0 0 1 0 1 1 1 0 1 0 1 0 1 1 0 1 1 1 0 0 1 0 1 0 1 1 0 0 0 0]


In [13]:
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import itertools
import matplotlib
from matplotlib import pyplot as plt

# compute a bunch of classification metrics 
def classification_metrics(y_true, y_pred, y_prob):
    cm_dict = {}
    cm_dict['Accuracy'] = accuracy_score(y_true, y_pred)
    cm_dict['Precision'] =  precision_score(y_true, y_pred)
    cm_dict['Recall'] =  recall_score(y_true, y_pred)
    cm_dict['F1'] =  f1_score(y_true, y_pred) 
    cm_dict['AUC'] = roc_auc_score(y_true, y_prob[:,0])
    cm_dict['Confusion Matrix'] = confusion_matrix(y_true, y_pred).tolist()
    return cm_dict

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """Plots a confusion matrix.
    Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    New BSD License - see appendix
    """
    cm_max = cm.max()
    cm_min = cm.min()
    if cm_min > 0: cm_min = 0
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm_max = 1
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    thresh = cm_max / 2.
    plt.clim(cm_min, cm_max)

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i,
                 round(cm[i, j], 3),  # round to 3 decimals if they are float
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    
cm_dict = classification_metrics(y_test_max, y_prob_max, y_probs)
for m in cm_dict:
    print(m, cm_dict[m])
cm = np.asarray(cm_dict['Confusion Matrix'])
plot_confusion_matrix(cm, ['fail','pass'], normalize=False)

Accuracy 0.875
Precision 0.8235294117647058
Recall 0.9333333333333333
F1 0.8749999999999999
AUC 0.04313725490196079
Confusion Matrix [[14, 3], [1, 14]]


<Figure size 640x480 with 2 Axes>